In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
import pandas_datareader.data as web

class FactorRiskModel:
    def __init__(self, tickers, weights):
        self.tickers = tickers
        self.weights = np.array(weights)
        self.data = None

    def fetch_data(self, start="2020-01-01"):
        # Download prices and compute monthly returns
        prices = yf.download(self.tickers, start=start, interval="1mo")['Close']
        returns = prices.pct_change().dropna()
        self.portfolio_returns = returns.dot(self.weights)
        
        # Fetch Fama-French 3 Factors from Ken French's library
        ff_factors = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start)[0]
        ff_factors = ff_factors / 100 # Convert to decimals
        
        # Align dates: convert YYYY-MM to datetime
        ff_factors.index = ff_factors.index.to_timestamp()
        self.combined = pd.merge(self.portfolio_returns.to_frame('Portfolio'), 
                                 ff_factors, left_index=True, right_index=True)
        
        # Calculate Excess Returns (Return - RiskFree)
        self.combined['Excess'] = self.combined['Portfolio'] - self.combined['RF']
        return self.combined

    def run_attribution(self):
        # Y = Excess Portfolio Returns, X = [Market, Size, Value]
        Y = self.combined['Excess']
        X = self.combined[['Mkt-RF', 'SMB', 'HML']]
        X = sm.add_constant(X)
        
        model = sm.OLS(Y, X).fit()
        print(model.summary())
        return model

# Example Usage: Swedish-US Mixed Portfolio
# Portfolio: Investor AB (INVE-B.ST), Evolution (EVO.ST), Apple (AAPL)
portfolio = FactorRiskModel(tickers=['INVE-B.ST', 'EVO.ST', 'AAPL'], weights=[0.4, 0.3, 0.3])
portfolio.fetch_data()
portfolio.run_attribution()

[*********************100%***********************]  3 of 3 completed


                            OLS Regression Results                            
Dep. Variable:                 Excess   R-squared:                       0.612
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     35.30
Date:                Sat, 21 Feb 2026   Prob (F-statistic):           8.40e-14
Time:                        13:11:51   Log-Likelihood:                 128.74
No. Observations:                  71   AIC:                            -249.5
Df Residuals:                      67   BIC:                            -240.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0061      0.005      1.220      0.2

C:\Users\fabre\AppData\Local\Temp\ipykernel_10428\2248181884.py:20: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factors = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start)[0]
C:\Users\fabre\AppData\Local\Temp\ipykernel_10428\2248181884.py:20: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factors = web.DataReader('F-F_Research_Data_Factors', 'famafrench', start=start)[0]
